In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

Populating the interactive namespace from numpy and matplotlib


---

## Семантический анализ твитов

Сегодня мы построим классификатор, который будет разделять текст на позитивные и негативные высказывания. Для этого мы воспользуемся уже размеченной базой.
Загрузим данные для анализа

In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')   # чтобы можно было оттуда считать файл с данными для этого задания

In [4]:
df = pd.read_excel('../data-samples/tweets_example.xlsx')
df.loc[16:25]

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
16,4.089069e+17,1386325965,fantanshik,"RT @Abdullin_A_R: @LikhodedovaMary эхх, а в УГ...",1,0,1,0,2074,82,44,5
17,4.089069e+17,1386325965,Tienn_En,"@marinaysol а, а то подумала, что у тебя там п...",1,0,0,0,6362,30,28,1
18,4.089069e+17,1386325966,bstrd666,@xLesherx @4EU3 зря вы с этой хуйней шутите)) ...,1,0,0,0,13431,473,111,2
19,4.089069e+17,1386325966,esken_h,@Moscow_advokat Очень главное спасибо for МЕ...,1,0,0,0,126795,581,86,10
20,4.089069e+17,1386325966,Obrazcova98,"У нас есть прекрасная история, как сдохнуть за...",1,0,0,0,492,14,23,0
21,4.089068e+17,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
22,4.089068e+17,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
23,4.089069e+17,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
24,4.089069e+17,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
25,4.089069e+17,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [6]:
df.columns

Index(['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav',
       'total_count', 'fol', 'friends', 'list_count'],
      dtype='object')

---

Все колонки таблицы могут содержать информацию о тональности твита, но мы будем ориентироваться исключительно на текст и на столбец отнесения к классу positiv.
Заменим значение -1 в колонке positive на 0

In [ ]:
df.positive[df.positive==-1]=0
df.loc[16:25]

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
16,408906854803451904,1386325965,fantanshik,"RT @Abdullin_A_R: @LikhodedovaMary эхх, а в УГ...",1,0,1,0,2074,82,44,5
17,408906854849204224,1386325965,Tienn_En,"@marinaysol а, а то подумала, что у тебя там п...",1,0,0,0,6362,30,28,1
18,408906855146983424,1386325966,bstrd666,@xLesherx @4EU3 зря вы с этой хуйней шутите)) ...,1,0,0,0,13431,473,111,2
19,408906857659392000,1386325966,esken_h,@Moscow_advokat Очень главное спасибо for МЕ...,1,0,0,0,126795,581,86,10
20,408906857982726080,1386325966,Obrazcova98,"У нас есть прекрасная история, как сдохнуть за...",1,0,0,0,492,14,23,0
21,408906762813579328,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,0,0,0,0,8064,111,94,2
22,408906818262687680,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",0,0,0,0,26,42,39,0
23,408906858515398720,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,0,0,0,0,718,49,249,0
24,408906914437685184,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",0,0,0,0,10628,207,200,0
25,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",0,0,0,0,35,17,34,0


-----

# Задание 1

1. С помощью *pd.read_csv()* загрузите датафреймы positive.csv и negative.csv (обратите внимание, что исходные таблицы не содержат наименования столбцов и на первой строке располагаются данные. Файлы расположены в папке datasets);
2. Объедините датафреймы с помощью *pd.concat()* в один датафрейм;
3. Убедитесь, что в новом датафрейме индексация сквозная и без повторов;
4. Переименуйте столбцы датафрейма (столбцы полностью соответствуют примеру);
5. Выведите информацию об общем количестве полученных твитов, сколько из них негативных, сколько позитивных.

-----

### Очистка и предобработка данных

Перед разработкой классификатора нам необходимо очистить и предобработать данные.

Начнем с очистки данных

----------------------------

***Внимание!*** Библиотека [*nltk*](https://www.nltk.org) может содержать не все компоненты. В случае возникновения ошибки необходимо запустить скрипт

*import nltk   
nltk.download()*

В открывшемся окне необходимо выбрать и установить требуемые компоненты

----------------------------

Приведем весь текст к строчным буквам:

In [ ]:
df.text = df.text.str.lower()
df.text.loc[19:22]

19    @moscow_advokat очень главное спасибо for   ме...
20    у нас есть прекрасная история, как сдохнуть за...
21    на работе был полный пиддес :| и так каждое за...
22    коллеги сидят рубятся в urban terror, а я из-з...
Name: text, dtype: object

---

Оставим в тексте только русские слова, удалив числа, знаки препинания, специальные символы и слова написанные латиницей:

In [ ]:
df.text = df.text.str.replace(r"[^А-Яа-я]"," ")
df.text.loc[19:22]

19                    очень главное спасибо       ме...
20    у нас есть прекрасная история  как сдохнуть за...
21    на работе был полный пиддес    и так каждое за...
22    коллеги сидят рубятся в               а я из з...
Name: text, dtype: object

---

Мы анализируем русскоязычный твиттер, поэтому английские слова, а так же числа, будут представлять частные случаи и формировать шум в данных. Но могут возникнуть задачи, где удаляемые слова и числа важны. В этом случае потребуется более взвешенный подход к очистке. Вам могут помочь [константы модуля *string*](https://docs.python.org/3/library/string.html)

Разобьем тексты на слова с помощью *word_tokenize*:

In [ ]:
from nltk.tokenize import word_tokenize
df.text = list(map(word_tokenize, df.text))
df.text.loc[19:22]

19    [очень, главное, спасибо, медвед, он, работал,...
20    [у, нас, есть, прекрасная, история, как, сдохн...
21    [на, работе, был, полный, пиддес, и, так, кажд...
22    [коллеги, сидят, рубятся, в, а, я, из, за, дол...
Name: text, dtype: object

---

В каждом языке имеются так называемые стоп-слова - это, например, предлоги, союзы, местоимения и т.д. Стоп-слова не несут смысловой нагрузки, но при этом встречаются достаточно часто. Существует множество словарей стоп-слов, мы воспользуемся словарем библиотеки *nltk*. При решении конкретных задач вы можете как расширить словарь стоп-слов, так и удалить из него любые слова.

In [ ]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
russian_stopwords

['а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

---

Удалим стоп-слова из наших данных

In [ ]:
def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df.text = list(map(delete_stopword, df.text))
df.text.loc[19:22]

19    [очень, главное, спасибо, медвед, работал, кло...
20              [прекрасная, история, сдохнуть, неделю]
21    [работе, полный, пиддес, каждое, закрытие, мес...
22    [коллеги, сидят, рубятся, долбанной, винды, могу]
Name: text, dtype: object

---

Проведем [лемматизацию](https://ru.wikipedia.org/wiki/Лемматизация) полученных слов

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df.text = list(map(lemmatization, df.text))
df.text.loc[19:22]

19    [очень, главный, спасибо, медведа, работать, к...
20              [прекрасный, история, сдохнуть, неделя]
21    [работа, полный, пиддес, каждый, закрытие, мес...
22     [коллега, сидеть, рубиться, долбать, винд, мочь]
Name: text, dtype: object

---

Теперь необходимо удалить все слова, которые встречаются только 1 раз

In [ ]:
from nltk.probability import FreqDist

def to_str(s):
    new_s = ' '.join(j for j in s)
    return new_s

text_tokens = word_tokenize(' '.join(j for j in list(map(to_str, df.text))))
text = nltk.Text(text_tokens)
fdist = FreqDist(text)
words_to_del = list(filter(lambda k: fdist[k] == 1, fdist))

def delete_word(words):
    global words_to_del
    new_s = [word for word in words if word not in words_to_del]
    return new_s

df.text = list(map(delete_word, df.text))
df.text = list(map(to_str, df.text))
df.text.loc[19:22]

19          очень работать
20                 история
21    работа полный каждый
22                  сидеть
Name: text, dtype: object

---

На этом очистка данных завершена. Можно ли утверждать, что очистка идеальна? Однозначно нет! Но, ее может оказаться достаточно для решения нашей задачи.   
Какие еще задачи могут возникнуть при очистке текстовых данных? Вот далеко неполный список:
- Обработка больших документов и больших коллекций текстовых документов, которые не помещаются в память.
- Извлечение текста из разметки, такой как HTML, PDF или другие структурированные форматы документов.
- Транслитерация символов с других языков.
- Декодирование символов Юникода в нормализованную форму, такую как UTF8
- Обработка доменных имен, фраз и сокращений.
- Обработка или удаление чисел, таких как даты и суммы.
- Поиск и исправление распространенных опечаток и ошибок в написании.

Можно очень долго заниматься очисткой и не достичь идеального результата. Лучше подойти к задаче итеративно - осуществить стандартную очистку и посмотреть на результат, если результат недостаточный, то провести дополнительные мероприятия по очистке.

In [ ]:
len(text_tokens), len(words_to_del), len(text_tokens) - len(words_to_del)

(262, 200, 62)

---

После очистки в наших данных осталось всего 62 слова (это не уникальные повторяющиеся слова, уникальных всего 29). Этого мало для построения классификаторов, но позволило существенно сократить время для знакомства с очисткой данных. В вашем проекте после очистки останется более 1,4 млн слов.


После очистки могут оказаться пустые твиты, т.е. эти твиты состояли из слов, записанных латиницей, стоп-слов, чисел, знаков припинания и уникальных слов. Такие твиты необходимо удалить из данных:

In [ ]:
len(df[df.text == '']), len(df[(df.text == '') & (df.positive == 1)])

(9, 6)

In [ ]:
df = df.drop(df[df.text == ''].index, axis = 0)
len(df)

33

---

После удаления пустых твитов у нас осталось 33 записи

----

# Задание 2

Произведите очистку данных, сформированных в задании 1. По результатам очистки выведите на экран следующую информацию:   
- Общее количество слов перед удалением слов, встречающихся 1 раз;
- Количество слов, встречающихся 1 раз;
- Итоговое количество слов;
- Количество пустых твитов;
- Из них позитивных твитов;
- Количество твитов после удаления пустых.

----

***Совет:*** сохраняйте промежуточные результаты очистки, чтобы в случае неверных действий на каком-либо этапе не пересчитывать все предыдущие этапы

---

Для разработки моделей нам необходимо оцифровать полученные данные. Мы воспользуемся двумя методами: мешком слов [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) и TF-IDF [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Но в начале необходимо разбить данные на обучающую и тестовую выборки.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.positive, test_size=0.2, random_state=21)

---

Рассмотрим количество твитов в выборке для обучения, из них позитивных, и в выборке для теста, из них позитивных

In [ ]:
len(y_train), y_train.sum(), len(y_test), y_test.sum()

(26, 13, 7, 2)

---

### Кодировка данных   

Кодируем наши данные мешком слов и tf-idf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

---

### Классификаторы

Построим классификатор с помощью логистической регрессии:   
на основе мешка слов:

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(cv_train, y_train)
cv_pred = lr.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, lr.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       1.00      0.40      0.57         5
           1       0.40      1.00      0.57         2

    accuracy                           0.57         7
   macro avg       0.70      0.70      0.57         7
weighted avg       0.83      0.57      0.57         7

train
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        13
           1       0.92      0.92      0.92        13

    accuracy                           0.92        26
   macro avg       0.92      0.92      0.92        26
weighted avg       0.92      0.92      0.92        26



---

на основе tf-idf:

In [ ]:
lr = LogisticRegression(random_state=21)
lr.fit(tfidf_train, y_train)
tfidf_pred = lr.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, lr.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       1.00      0.40      0.57         5
           1       0.40      1.00      0.57         2

    accuracy                           0.57         7
   macro avg       0.70      0.70      0.57         7
weighted avg       0.83      0.57      0.57         7

train
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       1.00      0.92      0.96        13

    accuracy                           0.96        26
   macro avg       0.96      0.96      0.96        26
weighted avg       0.96      0.96      0.96        26



---

Видно, что модели переобучены - это следствие малого количества данных.

---

Построим классификатор с помощью случайного леса [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html):   
на основе мешка слов:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=3, n_jobs=-1, random_state=21)
forest.fit(cv_train, y_train)
cv_pred = forest.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, forest.predict(cv_train)))

test
              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      0.50      0.67         2

    accuracy                           0.86         7
   macro avg       0.92      0.75      0.79         7
weighted avg       0.88      0.86      0.84         7

train
              precision    recall  f1-score   support

           0       0.87      1.00      0.93        13
           1       1.00      0.85      0.92        13

    accuracy                           0.92        26
   macro avg       0.93      0.92      0.92        26
weighted avg       0.93      0.92      0.92        26



---

на основе tf-idf:

In [ ]:
forest.fit(tfidf_train, y_train)
tfidf_pred = forest.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, forest.predict(tfidf_train)))

test
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         2

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7

train
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        13
           1       0.92      0.92      0.92        13

    accuracy                           0.92        26
   macro avg       0.92      0.92      0.92        26
weighted avg       0.92      0.92      0.92        26



---

Получили очень хороший результат. Надо проверить, получится ли так же хорошо на всем объеме данных.

---

# Задание 3

1. Кодировать данные методом мешка слов.
2. Кодировать данные методом TF-IDF.
3. Построить классификатор на основе логистической регрессии, используя мешок слов.
4. Построить классификатор на основе логистической регрессии, используя TF-IDF.
5. Построить классификатор на основе случайного леса, используя мешок слов.
6. Построить классификатор на основе случайного леса, используя TF-IDF.
7. Сделайте выводы о разработанных классификаторах.

---

При разбиении на обучающую и тестовую выборки, следует указать *test_size=0.3*

---

***Рекомендация:*** для случайного леса параметр n_estimator должен быть не менее 200

***Рекомендация:*** в чек-листе содержится объемный обучающий материал, поэтому лучше не затягивать с решением заданий